In [ ]:
import matplotlib
import matplotlib.pyplot as plt

## Знакомство с тензорами и автоматическое дифференцирование

In [ ]:
import torch

In [ ]:
torch.manual_seed(42)

x = torch.rand(3, 2)
print(x)
print(x.shape)
print()

y = torch.rand(3, 2)
x = x + y
print(x)

In [ ]:
import numpy as np

x = np.ones((3, 2), dtype=np.float32)
z = torch.from_numpy(x)

print(x)
print(z)

In [ ]:
x = torch.ones(3,2)
z = x.numpy()

print(x)
print(z)

$$F({\bf x})=\left|\left|{\bf x}\right|\right|^2$$
$$\frac{\partial F}{\partial x_i} = ?$$

In [ ]:
torch.manual_seed(42)

x = torch.rand(10, requires_grad=True)
print(x)
print()

norm = torch.dot(x, x)
print(norm)
print()

norm.backward()
print(x.grad)

## Классификаторы. Сети прямого распространения

In [ ]:
import numpy as np
import pandas as pd

names = ["length", "width", "size", "conc", "conc1", "asym", "m3long", "m3trans", "alpha", "dist", "class"]
data = pd.read_csv('magic04.csv', names=names)

x = np.asarray(data.iloc[:, :-1])
y = np.asarray(data.iloc[:, [-1]])
y = (y == 'g').astype(np.float32)

In [ ]:
import torch.utils.data

X = torch.from_numpy(x.astype(np.float32))
Y = torch.from_numpy(y.astype(np.float32))

dataset = torch.utils.data.TensorDataset(X, Y)
dataset_len = len(dataset)
train_dataset_len = int(0.8*dataset_len)
test_dataset_len = dataset_len - train_dataset_len

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_dataset_len, test_dataset_len])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset,  batch_size=5)

In [ ]:
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

INPUT_DIM  = x.shape[1]
HIDDEN_DIM = 20
OUTPUT_DIM = y.shape[1]

model = torch.nn.Sequential(
    torch.nn.Linear(INPUT_DIM, HIDDEN_DIM),
    torch.nn.Sigmoid(),
    torch.nn.Linear(HIDDEN_DIM, OUTPUT_DIM),
    torch.nn.Sigmoid(),
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 25
train_loss_hist = []
test_loss_hist = []
for e in tqdm(range(epochs)):
    for i, batch in enumerate(train_loader):
        features, labels = batch
        y_pred = model(features)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss_hist.append(loss)
    with torch.no_grad():
        count = 0
        total_loss = 0
        for i, batch in enumerate(test_loader):
            features, labels = batch
            y_pred = model(features)
            loss = loss_fn(y_pred, labels)
            count += 1
            total_loss += loss
        test_loss_hist.append(total_loss/count)

In [ ]:
train_loss_hist = np.asarray(train_loss_hist)
test_loss_hist = np.asarray(test_loss_hist)
plt.plot(train_loss_hist, '-*', label='Train loss')
plt.plot(test_loss_hist, '-*', label='Test loss')
plt.legend()
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
confusion_matrix = torch.zeros((2,2))
true_and_scores = []
with torch.no_grad():
    for batch in test_loader:
        features, labels = batch
        
        scores = model(features)
        y_pred = (scores > 0.5)
        for t, p in zip(labels.view(-1), y_pred.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

        true_and_scores.append(np.hstack([labels.numpy(), scores.numpy()]))
        
true_and_scores = np.vstack(true_and_scores)

tn, fp, fn, tp = confusion_matrix.view(-1)
accuracy  = (tp + tn) / (tn + fp + fn + tp)
precision = tp / (tp + fp)
recall    = tp / (tp + fn)
specificity = tn / (tn + fp)
baccuracy = 0.5 * (specificity + recall)
    
print("Accuracy                  = {:.4f}".format(accuracy))
print("Ballanced accuracy        = {:.4f}".format(baccuracy))
print("Precision (PPV)           = {:.4f}".format(precision))
print("Recall (sensitivity, TPR) = {:.4f}".format(recall))

In [ ]:
import sklearn.metrics

y_true = true_and_scores[:,0]
scores = true_and_scores[:,1]

min_score, max_score = np.min(scores), np.max(scores)
bins = np.linspace(min_score, max_score, 25)
plt.figure()
plt.hist(scores[y_true.reshape(-1) == 0], bins, alpha=0.5, label='Hadron (negative)')
plt.hist(scores[y_true.reshape(-1) == 1], bins, alpha=0.5, label='Gamma (positive)')
plt.xlabel("Decision function (value)")
plt.ylabel("Frequency")
plt.legend()

fpr, tpr, _ = sklearn.metrics.roc_curve(y_true, scores)
auc = sklearn.metrics.roc_auc_score(y_true, scores)
plt.figure()
plt.plot(fpr, tpr)
plt.title("Receiver operating characteristic")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
print("AUC                       = {:.4f}".format(auc))

**Задание 3a.1** Это задание на *бонусные балы*. Вам нужно использовать готовый код классификатора выше, чтобы улучшить качество классификатора в смысле *AUC*.

Попробуйте исследовать как результат меняется в зависимости от числа скрытых нейронов, вида функции активации, применяемого для обучения оптимизатора. Помогает ли добавление дополнительного скрытого слоя? Поможет ли применение анализа главных компонент для входных данных перед обучением нейросети?

*Бонусные баллы* положены за лучшие пять решений среди всей группы.

**Задание 3a.2** Перед вами набор данных рукописных цифр MNIST (черно-белые изображения, размером 28x28 пикселей каждое). Задача состоит в том, чтобы сделать автоэнкодер на базе свёрточной нейронной сети. Размерность пространства закодированного представления (латентный слой) должна быть 8. Подберите параметры свёрточных слоев.

*Бонусные баллы* положены за автоэнкодер с наименьшим среднеквадратичным отклонением `torch.nn.MSELoss` на тестовых данных и размерностью закодированного представления 8. Попробуйте добавить новые свёрточные слои или менять конфигурацию существующих, или вид функций активации.

In [ ]:
import torchvision
import torchvision.transforms as transforms

torch.manual_seed(43)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.0, 1.0)
])

train_dataset = torchvision.datasets.MNIST(root="./mnist", download=True, transform=transform, train=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root="./mnist", download=True, transform=transform, train=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True)
dataiter = iter(train_loader)
images, labels = dataiter.next()

plt.figure(figsize=(10, 7.5))
n_col = 4
for i in range(n_col):
    ax = plt.subplot(3, n_col, i + 1)
    plt.imshow(images[i].reshape(28,28), cmap='gray')

plt.show()

In [ ]:
torch.manual_seed(43)

INPUT_CH  = images.shape[1]
HIDDEN1_CH = # ???
HIDDEN2_CH = # ???
LATENT_DIM = 8

class CNNAutoEncoder(torch.nn.Module):
    def __init__(self):
        super(CNNAutoEncoder, self).__init__()

        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(INPUT_CH, HIDDEN1_CH, 3, stride=2),
            torch.nn.ReLU(True),
            
            torch.nn.Conv2d(HIDDEN1_CH, HIDDEN2_CH, 3, stride=2, bias=False),
            torch.nn.ReLU(True),
            
            torch.nn.Conv2d(HIDDEN2_CH, LATENT_DIM, 6, bias=False),
            torch.nn.Sigmoid(),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.LayerNorm((LATENT_DIM, 1, 1)),
            
            torch.nn.ConvTranspose2d(LATENT_DIM, HIDDEN2_CH, 6, bias=False),
            torch.nn.ReLU(True),
            
            torch.nn.ConvTranspose2d(HIDDEN2_CH, HIDDEN2_CH, 3, stride=2, bias=False),
            torch.nn.ReLU(True),
            
            torch.nn.ConvTranspose2d(HIDDEN2_CH, HIDDEN1_CH, 4, stride=2, bias=False),
            torch.nn.ReLU(True),
            
            torch.nn.ConvTranspose2d(HIDDEN1_CH, INPUT_CH, 1, bias=False),
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

model = CNNAutoEncoder()

loss_fn = torch.nn.MSELoss(reduction='mean')

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 5
train_loss_hist = []
test_loss_hist = []
for e in range(epochs):
    for i, batch in tqdm(enumerate(train_loader)):
        images, labels = batch
        _, decoded = model(images)
        loss = loss_fn(decoded, images)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
    train_loss_hist.append(loss)
    with torch.no_grad():
        count = 0
        total_loss = 0
        for i, batch in enumerate(test_loader):
            images, labels = batch
            _, decoded = model(images)
            loss = loss_fn(decoded, images)
            count += 1
            total_loss += loss
        test_loss_hist.append(total_loss/count)

In [ ]:
train_loss_hist = np.asarray(train_loss_hist)
test_loss_hist = np.asarray(test_loss_hist)
plt.plot(train_loss_hist, '-*', label='Train loss')
plt.plot(test_loss_hist, '-*', label='Test loss')
plt.legend()
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
count = 0
total_loss = 0
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        images, labels = batch
        _, decoded = model(images)
        loss = loss_fn(decoded, images)
        
        count += 1
        total_loss += loss

    print("Average loss = {:.4f}".format(total_loss/count))

**Задание 3a.3** Используя обученную модель автоэнкодера нарисуйте для четырех изображений тестового набора данных: само изображение, его закодированное представление в виде гистограммы, а так же раскодированное изображение. Ниже приведена заготовка кода рисующего изображения.

In [ ]:
plt.figure(figsize=(10, 7.5))
n_col = 4
for i in range(n_col):
    ax = plt.subplot(3, n_col, i + 1)
    inp = # ???
    plt.imshow(inp, cmap='gray')
    ax = plt.subplot(3, n_col, i + 1 + n_col)
    ecnoded = # ??? 
    plt.bar(np.arange(encoded[i].shape[0]), encoded[i])
    ax = plt.subplot(3, n_col, i + 1 + n_col * 2)
    decoded = # ???
    plt.imshow(decoded, cmap='gray')

plt.show()

**Задание 3a.4** Проделайте задание **3a.3**, но теперь на вход автоэнкодера следует подать изображения с добавленным к ним равномерно распределенным шумом амплитуды `noise_level`. После добавления шума, входные данные возможно придётся перенормировать, так как нейросеть ожидает, что в каждом пикселе содержится число от $0.0$ до $1.0$. Придумайте как это сделать.

In [ ]:
noise_level = 0.2

plt.figure(figsize=(10, 7.5))
n_col = 4
for i in range(n_col):
    ax = plt.subplot(3, n_col, i + 1)
    inp = # ???
    plt.imshow(inp, cmap='gray')
    ax = plt.subplot(3, n_col, i + 1 + n_col)
    ecnoded = # ??? 
    plt.bar(np.arange(encoded[i].shape[0]), encoded[i])
    ax = plt.subplot(3, n_col, i + 1 + n_col * 2)
    decoded = # ???
    plt.imshow(decoded, cmap='gray')

plt.show()

**Задание 3a.5** В задании **3a.4** вы должны были обнаружить, что автоэнкодер можно использовать для удаления шума из данных. В векторе `noise_levels` задана сетка амплитуды шумов для входных данных. Предлагается построить график зависимости от амплитуды шума для среднеквадратичного отклонения `torch.nn.MSELoss` между выходом автоэнкодера и незашумленным (истинным) входным изображением. Используйте полный тестовый датасет для вычисления значений. При шуме нулевой амплитуды (нет шума) среднеквадратичное отклонение будет определяться самой моделью. При постепенном добавлении шума он будет мешать нейросети распознать исходное изображение абсолютно корректно.

In [ ]:
noise_levels = np.linspace(0.0, 0.5, 10)
noise = []

for noise_level in noise_levels:
    count = 0
    total_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            images, labels = batch
            noised = (images + torch.randn(images.shape) * noise_level) / (1.0 + noise_level)
            _, decoded = model(noised)
            loss = loss_fn(decoded, images)
        
            count += 1
            total_loss += loss
        noise.append([total_loss/count])
        
plt.plot(noise_levels, np.asarray(noise), "-*")
plt.ylabel("MSE")
plt.xlabel("Noise level")
plt.show()

**Задание 3a.6** Аналогично заданию **3a.3** сгенерируйте и подайте на вход декодера случайные данные, совпадающие по своим статистическим свойствам с содержимым латентного слоя. Нарисуйте сгенерированные декодером изображения.

In [ ]:
plt.figure(figsize=(10, 7.5))
n_col = 4
for i in range(n_col):
    ax = plt.subplot(3, n_col, i + 1)
    encoded = # ???
    plt.bar(np.arange(encoded[i].shape[0]), encoded[i])
    ax = plt.subplot(3, n_col, i + 1 + n_col
    decoded = # ???
    plt.imshow(, cmap='gray')

**Задание 3a.7** Перед вами набор данных изображений STL10. Он состоит из цветных фотографий размером 96x96, отнесённых к одному из десяти различных классов: аэроплан, птичка, автомобиль, котик, олень, собака, лошадь, обезьяна, корабль, грузовик. Задача состоит в том, чтобы построить классификатор используя свёрточные нейронные сети, используя "transfer learning". Библиотека `torchvision` содержит в себе уже обученную глубокую свёрточную нейросеть AlexNet (`torchvision.models.alexnet`), эта сеть позволяет классифицировать фотографии на 1000 различных классов (набор данных ImageNet) и состоит из двух частей:
 * нескольких свёрточных слоев в начале сети
 * нескольких полносвязных слоев в конце сети
 
Можно считать, что первая половина сети генерирует некоторый набор универсальных признаков, на основе которого можно классифицировать фотографии из других наборов данных. Вам предлагается заменить часть нейросети состоящей из полносвязных слоев для решения задачи классификации STL10. Свёрточные слои обучать не следует, т.е. их веса остаются фиксированными и известными.

В качестве функции потерь для классификатора убодно использовать `torch.nn.CrossEntropyLoss`.

Используйте готовую функцию `sklearn.metrics.confusion_matrix` и класс `sklearn.metrics.ConfusionMatrixDisplay` чтобы вычислить и нарисовать матрицу ошибок для тестового набора данных. Используйте готовую функцию `sklearn.metrics.classification_report`, чтобы подсчитать точность (precision) и полноту (recall) для каждого из целевых классов.

*Бонусные балы* полагаются за лучшее среди всей группы значение функции потерь на тестовом наборе данных.

In [ ]:
import torch
import torchvision.models
import torchvision.datasets
import torchvision.transforms

def show(img):
    npimg = img.numpy()
    plt.figure(figsize=(10,8))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    plt.show()

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    normalize])

train_dataset = torchvision.datasets.STL10("./stl10", split='train', transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = torchvision.datasets.STL10("./stl10", split='test', transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True)

class_names = np.array(["airplane", "bird", "car", "cat", "deer", "dog", "horse", "monkey", "ship", "truck"])

dataiter = iter(train_loader)
images, labels = dataiter.next()

print(class_names[labels.numpy()])
show(torchvision.utils.make_grid(images, normalize=True))

In [ ]:
import torchvision.models

alexnet = torchvision.models.alexnet(pretrained=True)
for param in alexnet.parameters():
    param.requires_grad = False

In [ ]:
torch.manual_seed(43)

HIDDEN_DIM = # ???

class ClassifierModel(torch.nn.Module):
    def __init__(self):
        super(ClassifierModel, self).__init__()
        
        self.features = alexnet.features
        self.avgpool = alexnet.avgpool
        self.classifier = torch.nn.Sequential(
            nn.Dropout(),
            # тут должна была быть полносвязная сеть, но она пропала.
            # известно, что на входе должно быть 256 * 6 * 6 значений,
            # а на выходе вектор из 10 признаков
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
model = ClassifierModel()

loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 1e-3
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

epochs = 5
for e in range(epochs):
    for i, batch in tqdm(enumerate(train_loader)):
        images, labels = batch
        output = model(images)
        loss = loss_fn(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("epoch = {} batch = {} loss = {}".format(e, i, loss))

In [ ]:
count = 0
total_loss = 0
true_and_pred = []
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        images, labels = batch
        output = model(images)
        loss = loss_fn(output, labels)
        
        count += 1
        total_loss += loss
        
        y_pred = torch.max(output, axis=1).indices
        true_and_pred.append(np.vstack([labels.numpy(), y_pred.numpy()]))

true_and_pred = np.hstack(true_and_pred)

print("Average loss = {:.4f}".format(total_loss/count))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

#print(classification_report(???))

cm = # confusion_matrix ???

cmd = ConfusionMatrixDisplay(cm, display_labels=class_names)
_ = cmd.plot()